In [23]:
import numpy as np
import pandas as pd
import matplotlib as mpl

In [24]:
df_original = pd.read_csv("reordered2016crashes.csv")

In [25]:
df_original

,CRASH_CRN,DISTRICT,CRASH_COUNTY,MUNICIPALITY,POLICE_AGCY,CRASH_YEAR,CRASH_MONTH,DAY_OF_WEEK,TIME_OF_DAY,HOUR_OF_DAY,...,ACCESS_CTRL,ROADWAY_COUNTY,LANE_COUNT,RDWY_ORIENT,ROAD_OWNER,ROUTE,SPEED_LIMIT,SEGMENT,OFFSET,STREET_NAME
0,2016016958,11,2,2301,2301,2016,2,6,1255,12,...,0,2,2,N,4,0,25,0,0,10TH ST BR
1,2016017490,11,2,2301,2301,2016,2,7,1752,17,...,0,2,2,E,4,0,25,0,0,SARAH ST
2,2016047300,11,2,2121,68D02,2016,4,2,2321,23,...,1,2,2,N,2,28,55,320,588,ALLEGHENY VALLEY EX
3,2016048852,11,2,2301,2301,2016,5,6,2340,23,...,0,2,2,W,4,0,25,0,0,SPROAT WY
4,2016052699,11,2,2301,2301,2016,5,1,1450,14,...,0,2,3,U,4,0,25,0,0,E GENERAL ROBINSON ST
5,2016073947,11,2,2102,2102,2016,7,6,843,8,...,1,2,1,W,2,8005,0,250,0,RAMP A RD
6,2016011851,11,2,2301,2301,2016,1,6,1320,13,...,0,2,2,E,4,0,25,0,0,SHETLAND ST
7,2016017595,11,2,2112,2112,2016,2,1,1000,10,...,0,2,2,N,4,0,30,0,0,DORSEYVILLE RD
8,2016024918,11,2,2301,2301,2016,3,5,1740,17,...,0,2,2,W,4,0,25,0,0,BAILEY AV
9,2016036866,11,2,2446,2446,2016,3,6,2130,21,...,0,2,2,W,4,0,35,0,0,MALL BL


In [26]:
category_time = ["CRASH_YEAR", "CRASH_MONTH", "TIME_OF_DAY", "HOUR_OF_DAY"]
category_conditions = ["ILLUMINATION", "WEATHER", "ROAD_CONDITION"]
category_type = ["COLLISION_TYPE", "URBAN_RURAL"]
category_people_count = ["PERSON_COUNT", "VEHICLE_COUNT"] 
category_injury = ["FATAL_COUNT", "INJURY_COUNT", "MAJ_INJ_COUNT", "MOD_INJ_COUNT", "MIN_INJ_COUNT", "UNK_INJ_DEG_COUNT", "MAX_SEVERITY_LEVEL"]
category_belt = ["UNBELTED", "UNBELTED_OCC_COUNT", "UNB_DEATH_COUNT", "UNB_MAJ_INJ_COUNT", "BELTED_DEATH_COUNT", "BELTED_MAJ_INJ_COUNT"]
category_age = ["DRIVER_COUNT_16YR", "DRIVER_COUNT_17YR", "DRIVER_COUNT_18YR", "DRIVER_COUNT_19YR", "DRIVER_COUNT_20YR", "DRIVER_COUNT_50_64YR", "DRIVER_COUNT_65_74YR", "DRIVER_COUNT_75PLUS"]
category_location = ["LATITUDE", "LONGITUDE", "DEC_LAT", "DEC_LONG", "STREET_NAME"]
category_alcohol = ["ALCOHOL_RELATED", "DRINKING_DRIVER", "UNDERAGE_DRNK_DRV", "IMPAIRED_DRIVER"]
category_attention = ["CELL_PHONE", "DISTRACTED", "FATIGUE_ASLEEP"]
category_speed = ["LIMIT_65MPH", "SPEEDING", "SPEEDING_RELATED", "AGGRESSIVE_DRIVING"]


categories_all = category_time + category_conditions + category_type + category_people_count + \
                category_injury + category_belt + category_age + category_location + \
                category_alcohol + category_attention + category_speed

df_trimmed = df_original[categories_all]

In [27]:
df_trimmed

,CRASH_YEAR,CRASH_MONTH,TIME_OF_DAY,HOUR_OF_DAY,ILLUMINATION,WEATHER,ROAD_CONDITION,COLLISION_TYPE,URBAN_RURAL,PERSON_COUNT,...,DRINKING_DRIVER,UNDERAGE_DRNK_DRV,IMPAIRED_DRIVER,CELL_PHONE,DISTRACTED,FATIGUE_ASLEEP,LIMIT_65MPH,SPEEDING,SPEEDING_RELATED,AGGRESSIVE_DRIVING
0,2016,2,1255,12,1,1,0,4,3,5,...,0,0,0,0,0,0,0,1,1,1
1,2016,2,1752,17,4,4,3,4,3,3,...,0,0,0,0,0,0,0,0,0,0
2,2016,4,2321,23,3,1,0,7,3,4,...,0,0,0,0,1,0,0,0,1,1
3,2016,5,2340,23,3,1,0,7,3,1,...,1,0,1,0,0,0,0,0,0,0
4,2016,5,1450,14,1,1,0,4,3,2,...,0,0,0,0,0,0,0,0,0,1
5,2016,7,843,8,1,1,0,4,3,2,...,0,0,0,0,0,0,0,0,0,1
6,2016,1,1320,13,1,1,6,1,3,3,...,0,0,0,0,0,0,0,0,1,1
7,2016,2,1000,10,1,1,1,7,3,1,...,0,0,0,0,0,0,0,0,0,0
8,2016,3,1740,17,4,1,0,1,3,1,...,0,0,0,0,0,1,0,0,0,0
9,2016,3,2130,21,3,1,0,0,3,1,...,0,0,0,0,0,0,0,0,0,0
